In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
   # for filename in filenames:
     #   print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df= pd.read_csv('/kaggle/input/job-recommendation-datasets/Combined_Jobs_Final.csv')
df.head(3) # understanding first 3 rows.

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/job-recommendation-datasets/Combined_Jobs_Final.csv'

In [ ]:
df.info() # info that what all columns are about

In [ ]:
# We will use Content-based recommendation system :- It uses tags, descriptions
# we will be using only 2 columns Title, Job Description column.
df_old=df[['Title','Job.Description']]
df_old # df dataframe will be our new dataset and we will be working on this dataset only.

In [ ]:
# can reduce no. of rows so that it can work on hardware
#df=df_old.sample(n=1000,random_state=0)  # out of 84089 rows will only consider  1000 rows
# will get a new different set of DataFrame with 1000 random rows Every time when running the code
#df=df.iloc[:10000, :]
df=df_old.head(10000).copy()   # Using head for selection, then copy into new dataframe, so that it don't create error in applying apply() at later point

In [ ]:
# will need to explore both column one by one
# as these two columns are "Text" and  
# our machine can't understand text so first of all we need to clean our dataset
df['Title']

In [ ]:
#lots of job Kitchen Staff,Book Keeper,....

In [ ]:
df['Title'][6558]  # row 1

In [ ]:
df['Job.Description']

In [ ]:
df['Job.Description'][6558] # information about column 1 Job.Description

In [ ]:
# In this in each row in entire Description, we can find a lots of keywords in decription
# we will first clean content of both column 'Title','Job.Description'
# which means we will remove all other things other than text.
# Like STOP WORDS, Special Characters , \r\n\r\n,  other clutters other than text.

In [ ]:
# We will make a function which will clean all these non-essential content from our Title and Description.

import nltk # Natural Language Toolkit
from nltk.stem.porter import PorterStemmer  # It is very useful algorithm 
from nltk.corpus import stopwords

import re # regular expression library from python

In [ ]:
# making object of PorterStemmer
ps=PorterStemmer()
# Porter Stemmer is an algorithm used in natural language processing (NLP) for stemming words

In [ ]:
# making a function to clean The text of Title and Description
def cleaning(txt):
    cleaned_txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    return cleaned_txt    
    # we don't want \n \r and special characters, We want text and Integers only and other things will be removed
    # we used Substract function  sub() of Regular Expression library(re)
   # Inside that using "" or'' if content is other than i.e. (Negation of)^ a-z , A-Z or 0-9 and whitespace characters (\s)
# other than these if any other character encounters so don't include them and replace them with '' empty string.

In [ ]:
cleaning('This is my book with 0987 3 pages and digit  \n]\r\t $@$%')
# we seen above function removed ] and other special characters, but will not able to remove \r\n\t,

In [ ]:
# correcting this function to clean The text of Title and Description
def cleaning(txt):
    cleaned_txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    # we don't want \n \r, We want text and Integers only and other things will be removed
    # we used Substract function  sub() of Regular Expression library(re)
   # Inside that using "" or'' if content is other than i.e. (Negation of)^ a-z , A-Z or 0-9 
# other than these if any other character encounters so don't include them and replace them with '' empty string.
# But this can remove special characters, but will not able to remove \r\n\t, we will tokenise
    tokens= nltk.word_tokenize(cleaned_txt.lower())
# we will tokenise(we will break word-by-word) and will also convert all characters into lower case
    return tokens

In [ ]:
cleaning('This is my book with 0987  3 pages and  digit \n]\r\t $@$%')
# tokenized all and converted to lower case, this will also remove \n \r \t,.. in the process of tokenization

In [ ]:
# now we don't want tokens like 'this', 'is', 'my', 'with','and'
# 'copy','0987', '3', 'pages' are good keyword can use

# so will remove stopWords, Stopwords are the most frequently occurring words in a language 
# that carry little or no meaning on their own. They include words like "the," "a," "an," "is," "of," "and," etc.
# and will use Stemming by  ps=PorterStemmer() function
# Stemming aims to reduce words to their base or root form.
#For example, "running," "runs," and "ran" would all be stemmed to "run."
# The goal is to group similar words together, focusing on their core meaning.

In [ ]:
# correcting this function to clean The text of Title and Description by also removing StemWords
def cleaning(txt):
    cleaned_txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    tokens= nltk.word_tokenize(cleaned_txt.lower())
    # using list comprehension 
    # [for word in tokens if word not in stopwords.words('english')]
    # let say tokens have 1,00,000 words so above line will run that number of time
    #if these words are not from  then will take  those words in ps.stem() function
    stemming = [ps.stem(word) for word in tokens if word not in stopwords.words('english')]

    
    return stemming    

In [ ]:
cleaning('This is my book with 0987  3 pages and  digit moving driven \n]\r\t $@$%')
# [for word in tokens if word not in stopwords.words('english')] taken only those words who are not Stopwords
# these are stored in word and then given to ps.stem(word)
# this makes those non-stopwords into base form by help of ntlk PorterStemmer() function from nltk.stem.porter library by  import PorterStemmer 

In [ ]:
# it is returning a list,  returning it as string

In [ ]:
# correcting this function to clean The text of Title and Description by making change in return line
def cleaning(txt):
    cleaned_txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    tokens= nltk.word_tokenize(cleaned_txt.lower())
    
    stemming = [ps.stem(word) for word in tokens if word not in stopwords.words('english')]

    
    #return stemming    
    return " ".join(stemming) # joining tokens by separating them with " " space 

In [ ]:
cleaning('This is my book with 0987  3 pages and  digit moving driven \n]\r\t $@$%')
# converted into a sigle string

In [ ]:
# Now we seen that it can process on paragraph and find some keywords
# now we will pass our title in this function
df['Title']= df['Title'].apply( lambda x:cleaning(x))
# as Title having 84089 so we will apply lambda, 
# and x will take each row of Title one by one, and will send it to cleaning() function.

It can give error if our all the data is not of object type, but fortunately we have all data as object.

In [ ]:
df['Title']

In [ ]:
# df['Job.Description']= df['Job.Description'].apply( lambda x:cleaning(x))
# above line gives error that  "expected string or bytes-like object"
# so before apply() function we will convert our column into string type
df['Job.Description']= df['Job.Description'].astype(str).apply( lambda x:cleaning(x))
# using astype(str) to convert our column into string

In [ ]:
df['Job.Description']

In [ ]:
df['Job.Description'][6558] # now if we just see again row 1 does not contain unnecessary tokens or special characters

In [ ]:
df['new_col']= df['Title'] +" " + df['Job.Description']
# making a new 3rd column by joining the old two columns

In [ ]:
df.head(3)

In [ ]:
# now talking about vectorization
# as we are maikng an algorithm for Recommendation system
# machine can understand numbers and not text
# so first we will try to make vector from it.

# 2 things Vector , tfidfvectorizer
# we find cosine angle between two vectors if it comes 0, so vectors are too similar and if it comes 90 then these are totally different 
# sckit-learn provides us cosine
# but one more thing need to be done for these vectors-> tfidfvectorizer
# tfidfvectorizer -> Term Frequency-Inverse Document Frequency Vectorizer
# our row(84,000+) which is containing sentences are also called document.
# tfidfvectorizer will help in calculating frequency of each word/token in entire dataset

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# object of TfidfVectorizer
tfidf= TfidfVectorizer()
#using fit_transform() function 
matrix=tfidf.fit_transform(df['new_col'])
# out of these rows we will find most similar rows/vector by help of cosine_similarity given by scikit learn
similarity= cosine_similarity(matrix)
similarity  
# this will return lists containing values such that it describes similarity of one row values with other remaining rows values

In [ ]:
similarity[3]
# this give a list showing similarity score all rows which are similar to this row 3 (may be similar tag, content)

sorted(list(enumerate(similarity[idx])), key=lambda x:x[1] , reverse=False)

**enumerate** is a built-in function in Python that takes an iterable (like a list) and returns an iterator that produces tuples.Each tuple contains two elements:
* The first element is the index (position) of the item in the original iterable.
* The second element is the item itself.

**similarity[idx]** refers to a specific element (indexed by idx) within a list or array named similarity.
It sorts the list of tuples created by enumerate(similarity[idx]) based on the similarity values (the second element in each tuple) in ascending order (reverse=False).

**list(...)** converts the iterator returned by enumerate into a regular Python list. This list contains the tuples with indices and corresponding similarity values.

**sorted()** function sorts the elements within the list. However, it requires a sorting criteria, which is specified by the key argument.

**key=lambda x: x[1], reverse=False** 
     lambda x: x[1] is a **lambda function** that takes a single argument x (which represents one of the tuples in the list). It returns the second element (x[1]) of the tuple, which is the similarity value
**reverse=False** sort the list based on the similarity values (second element) in ascending order

In [ ]:
# function which takes which takes a title input by user
def recommendation(title):
    idx= df[df['Title'] == title].index[0]  # storing index or rows which having same Title as given by user(title)
    #idx=df.index.get_loc(idx) # when indexing is not linearly selected
    y= sorted(list(enumerate(similarity[idx])), key=lambda x:x[1], reverse=False) #Sort similarity with indices
    top_20= y[1:20]  # Select top 20 similar elements from index 1 (inclusive) to 20 (exclusive)
    
    jobs=[] # try to store value in list
    for idx in top_20:
        jobs.append(df.iloc[idx[0]].Title) # storing all 20 similar title in a list one by one
    return jobs

In [ ]:
recommendation('medic front offic officeteam')

In [ ]:
# creating simple website for this recommendation system
# need to pickle 2 things
import pickle
pickle.dump(df, open('df.pkl','wb')) # binary mode as write-binary mode(wb)
pickle.dump(similarity,open('similarity.pkl','wb')) 
# created these 2 pickle files 'df.pkl' and 'similarity.pkl' can be used later in website
# these 2 files 'df.pkl' and 'similarity.pkl'  are being formed in our Output folder

40
Web apps. brings us to deploying machine learning models as web applications. The traditional approach is to wrap the API via the use of web frameworks such as Django and Flask. A much simpler approach is to use a low-code solution such as Streamlit to create a web app. Streamlit is an open-source Python library that makes it easy to create and share web apps for machine learning and data science. 

!pip install streamlit
import streamlit as pd
import pandas as pd
import pickle 

df = pickle.load(open('df.pkl', 'rb'))  # binary mode as read-binary mode(rb) 
df = pickle.load(open('similarity.pkl', 'rb'))

# streamlit will help in creating website
# web app 
st.title(